# Preprocessing Student Answer
OCR all student's answer

In [3]:
pdf_file = "../data/TestScript.pdf"

In [4]:
import os

file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]
base_path = "../marking_form/" + file_name
base_path_images = base_path + "/images/"
base_path_annotations = base_path+"/annotations/"
base_path_questions = base_path+"/grading_form/questions"
base_path_javascript = base_path+"/grading_form/javascript"

# create directory tree for base_path_images
os.makedirs(base_path_images, exist_ok=True)
os.makedirs(base_path_annotations, exist_ok=True)
os.makedirs(base_path_questions, exist_ok=True)
os.makedirs(base_path_javascript, exist_ok=True)

In [5]:
import json
annotations_path = base_path_annotations + "annotations.json"
with open(annotations_path, "r") as f: 
    annotations = json.load(f) 

annotations

{'0': [{'x': 238, 'y': 173, 'width': 322, 'height': 77, 'label': 'NAME'},
  {'x': 175, 'y': 259, 'width': 471, 'height': 83, 'label': 'ID'},
  {'x': 835, 'y': 248, 'width': 168, 'height': 91, 'label': 'CLASS'},
  {'x': 164, 'y': 344, 'width': 322, 'height': 111, 'label': 'Q1'}],
 '1': [{'x': 184, 'y': 482, 'width': 347, 'height': 138, 'label': 'Q36'},
  {'x': 665, 'y': 460, 'width': 410, 'height': 150, 'label': 'Q37'},
  {'x': 1166, 'y': 473, 'width': 384, 'height': 143, 'label': 'Q38'}]}

In [6]:
#flatten annotations to list 
annotations_list = []
for page in annotations:
    for annotation in annotations[page]:
        annotation["page"] = int(page)
        annotations_list.append(annotation) 
annotations_list

[{'x': 238, 'y': 173, 'width': 322, 'height': 77, 'label': 'NAME', 'page': 0},
 {'x': 175, 'y': 259, 'width': 471, 'height': 83, 'label': 'ID', 'page': 0},
 {'x': 835, 'y': 248, 'width': 168, 'height': 91, 'label': 'CLASS', 'page': 0},
 {'x': 164, 'y': 344, 'width': 322, 'height': 111, 'label': 'Q1', 'page': 0},
 {'x': 184, 'y': 482, 'width': 347, 'height': 138, 'label': 'Q36', 'page': 1},
 {'x': 665, 'y': 460, 'width': 410, 'height': 150, 'label': 'Q37', 'page': 1},
 {'x': 1166, 'y': 473, 'width': 384, 'height': 143, 'label': 'Q38', 'page': 1}]

In [7]:
# extract list of label from annotations as questions
questions = []
for annotation in annotations_list:
    if annotation["label"] not in questions:
        questions.append(annotation["label"])
# remove 'NAME', 'ID', 'CLASS' if exists in questions
if 'NAME' in questions:
    questions.remove('NAME')
if 'ID' in questions:
    questions.remove('ID')
if 'CLASS' in questions:
    questions.remove('CLASS')    

# sort questions 
questions.sort()
questions = ['NAME', 'ID', 'CLASS'] + questions
questions

['NAME', 'ID', 'CLASS', 'Q1', 'Q36', 'Q37', 'Q38']

Check for the regeneration of question.

In [8]:
import os
import json

questionAndControl = {}
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file == "control.json":
            question = path[len(base_path_questions) + 1 :]
            f = open(os.path.join(path, file))
            data = json.load(f)
            if "regenerate" in data:
                questionAndControl[question] = data
            f.close()

questionAndControl

{}

In [13]:
from distutils.dir_util import copy_tree
import shutil
import os

from_directory = os.path.join(os.getcwd(), "..","templates", "javascript")
copy_tree(from_directory, base_path_javascript)
ico = os.path.join(os.getcwd(), "..","templates", "favicon.ico")
# copy ico file  to base_path
shutil.copyfile(ico, base_path+"/favicon.ico")

'../marking_form/TestScript/favicon.ico'

Generate the index.html

In [2]:
from pathlib import Path
from jinja2 import Environment, FileSystemLoader

file_loader = FileSystemLoader("../templates")
env = Environment(loader=file_loader)
template = env.get_template("index.html")

output = template.render(
    studentsScriptFileName=file_name,
    textAnswer=questions,
    optionAnswer=[],
)
# open text file
path = Path(os.path.join(base_path, "index.html"))
text_file = open(path, "w")
text_file.write(output)
text_file.close()

# Start Python HTTPServer

The webserver log is in output/server.log.

If you are in development and don't want the notebook being blocked by running webserver, you can open a terminal and run the below 2 commands.

python server.py 8000